In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_squared_error
from math import sqrt
import os
import pickle
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
data = pd.read_csv('Environment_Temperature_change_E_All_Data.csv', encoding='latin1')
data

,Area Code,Area Code (M49),Area,Months Code,Months,Element Code,Element,Unit,Y1961,Y1961F,...,Y2019,Y2019F,Y2020,Y2020F,Y2021,Y2021F,Y2022,Y2022F,Y2023,Y2023F
0,2,'004,Afghanistan,7001,January,7271,Temperature change,°c,0.745,E,...,2.911,E,-0.547,E,0.701,E,2.458,E,-2.200,E
1,2,'004,Afghanistan,7001,January,6078,Standard Deviation,°c,1.954,E,...,1.954,E,1.954,E,1.954,E,1.954,E,1.954,E
2,2,'004,Afghanistan,7002,February,7271,Temperature change,°c,-1.787,E,...,0.147,E,2.544,E,4.220,E,2.502,E,3.697,E
3,2,'004,Afghanistan,7002,February,6078,Standard Deviation,°c,2.591,E,...,2.591,E,2.591,E,2.591,E,2.591,E,2.591,E
4,2,'004,Afghanistan,7003,March,7271,Temperature change,°c,0.445,E,...,0.360,E,0.664,E,2.625,E,3.756,E,3.895,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9651,5873,'198,OECD,7018,JunJulAug,6078,Standard Deviation,°c,0.248,E,...,0.248,E,0.248,E,0.248,E,0.248,E,0.248,E
9652,5873,'198,OECD,7019,SepOctNov,7271,Temperature change,°c,0.039,E,...,1.214,E,1.376,E,1.763,E,1.289,E,2.254,E
9653,5873,'198,OECD,7019,SepOctNov,6078,Standard Deviation,°c,0.378,E,...,0.378,E,0.378,E,0.378,E,0.378,E,0.378,E
9654,5873,'198,OECD,7020,Meteorological year,7271,Temperature change,°c,0.169,E,...,1.291,E,1.431,E,1.589,E,1.237,E,1.706,E


In [3]:
columns_to_remove = data.columns[data.columns.str.endswith('F')]

# Drop the columns ending with 'F'
data = data.drop(columns=columns_to_remove)
data = data.drop( columns = ['Unit','Element Code','Area Code (M49)'])
# Display the modified DataFrame
print(data)

      Area Code         Area  Months Code               Months  \
0             2  Afghanistan         7001              January   
1             2  Afghanistan         7001              January   
2             2  Afghanistan         7002             February   
3             2  Afghanistan         7002             February   
4             2  Afghanistan         7003                March   
...         ...          ...          ...                  ...   
9651       5873         OECD         7018          JunJulAug   
9652       5873         OECD         7019          SepOctNov   
9653       5873         OECD         7019          SepOctNov   
9654       5873         OECD         7020  Meteorological year   
9655       5873         OECD         7020  Meteorological year   

                 Element  Y1961  Y1962  Y1963  Y1964  Y1965  ...  Y2014  \
0     Temperature change  0.745  0.015  2.706 -5.250  1.854  ...  1.716   
1     Standard Deviation  1.954  1.954  1.954  1.954  1.9

In [4]:
data.columns

Index(['Area Code', 'Area', 'Months Code', 'Months', 'Element', 'Y1961',
       'Y1962', 'Y1963', 'Y1964', 'Y1965', 'Y1966', 'Y1967', 'Y1968', 'Y1969',
       'Y1970', 'Y1971', 'Y1972', 'Y1973', 'Y1974', 'Y1975', 'Y1976', 'Y1977',
       'Y1978', 'Y1979', 'Y1980', 'Y1981', 'Y1982', 'Y1983', 'Y1984', 'Y1985',
       'Y1986', 'Y1987', 'Y1988', 'Y1989', 'Y1990', 'Y1991', 'Y1992', 'Y1993',
       'Y1994', 'Y1995', 'Y1996', 'Y1997', 'Y1998', 'Y1999', 'Y2000', 'Y2001',
       'Y2002', 'Y2003', 'Y2004', 'Y2005', 'Y2006', 'Y2007', 'Y2008', 'Y2009',
       'Y2010', 'Y2011', 'Y2012', 'Y2013', 'Y2014', 'Y2015', 'Y2016', 'Y2017',
       'Y2018', 'Y2019', 'Y2020', 'Y2021', 'Y2022', 'Y2023'],
      dtype='object')

In [5]:
data = data[data['Element'] != 'Standard Deviation']

# Reset index after dropping rows
data.reset_index(drop=True, inplace=True)

# Display the modified DataFrame
print(data)

      Area Code         Area  Months Code               Months  \
0             2  Afghanistan         7001              January   
1             2  Afghanistan         7002             February   
2             2  Afghanistan         7003                March   
3             2  Afghanistan         7004                April   
4             2  Afghanistan         7005                  May   
...         ...          ...          ...                  ...   
4823       5873         OECD         7016          DecJanFeb   
4824       5873         OECD         7017          MarAprMay   
4825       5873         OECD         7018          JunJulAug   
4826       5873         OECD         7019          SepOctNov   
4827       5873         OECD         7020  Meteorological year   

                 Element  Y1961  Y1962  Y1963  Y1964  Y1965  ...  Y2014  \
0     Temperature change  0.745  0.015  2.706 -5.250  1.854  ...  1.716   
1     Temperature change -1.787  2.390  3.875 -0.218 -0.1

In [6]:
data = data.drop( columns = ['Element','Months Code','Area Code'])
data

,Area,Months,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,Y1968,...,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019,Y2020,Y2021,Y2022,Y2023
0,Afghanistan,January,0.745,0.015,2.706,-5.250,1.854,3.796,-1.380,0.376,...,1.716,2.017,3.594,1.367,2.111,2.911,-0.547,0.701,2.458,-2.200
1,Afghanistan,February,-1.787,2.390,3.875,-0.218,-0.100,3.529,0.176,-2.071,...,-3.020,2.696,2.448,0.000,2.774,0.147,2.544,4.220,2.502,3.697
2,Afghanistan,March,0.445,1.268,0.365,1.632,-0.923,-0.010,-0.649,0.470,...,-0.073,-0.465,2.468,0.994,4.632,0.360,0.664,2.625,3.756,3.895
3,Afghanistan,April,-1.786,0.040,0.859,-0.579,-1.850,-1.101,-1.369,-0.633,...,0.661,1.755,1.009,1.426,1.498,0.962,0.084,2.275,4.851,1.747
4,Afghanistan,May,1.351,-0.160,-0.727,-0.169,-0.190,-0.182,-1.241,-1.860,...,1.035,2.030,3.412,3.482,0.926,0.597,1.198,2.329,2.368,1.391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4823,OECD,DecJanFeb,0.569,-0.355,-0.202,0.106,-0.727,-0.272,-0.160,-0.009,...,0.709,1.343,2.689,1.940,1.475,1.539,2.105,2.081,0.973,1.618
4824,OECD,MarAprMay,-0.035,-0.185,0.148,-0.588,-0.104,-0.080,-0.339,0.289,...,0.796,1.197,1.933,1.244,1.239,1.346,0.938,1.125,1.187,1.210
4825,OECD,JunJulAug,0.105,0.054,-0.042,-0.107,-0.492,-0.013,-0.180,-0.445,...,0.789,0.950,1.314,1.083,1.083,1.062,1.305,1.387,1.499,1.743
4826,OECD,SepOctNov,0.039,0.467,0.673,-0.146,-0.193,-0.291,0.328,0.217,...,0.998,1.679,1.535,1.204,0.577,1.214,1.376,1.763,1.289,2.254


In [7]:
melted_df = pd.melt(data, id_vars=['Area', 'Months'], var_name='Year', value_name='Temperature')

# Display the reshaped DataFrame
print(melted_df)

               Area               Months   Year  Temperature
0       Afghanistan              January  Y1961        0.745
1       Afghanistan             February  Y1961       -1.787
2       Afghanistan                March  Y1961        0.445
3       Afghanistan                April  Y1961       -1.786
4       Afghanistan                  May  Y1961        1.351
...             ...                  ...    ...          ...
304159         OECD          DecJanFeb  Y2023        1.618
304160         OECD          MarAprMay  Y2023        1.210
304161         OECD          JunJulAug  Y2023        1.743
304162         OECD          SepOctNov  Y2023        2.254
304163         OECD  Meteorological year  Y2023        1.706

[304164 rows x 4 columns]


In [8]:
melted_df['Year'] = melted_df['Year'].str.replace('Y', '')

# Display the DataFrame after removing 'Y' from the 'Year' column
print(melted_df)


               Area               Months  Year  Temperature
0       Afghanistan              January  1961        0.745
1       Afghanistan             February  1961       -1.787
2       Afghanistan                March  1961        0.445
3       Afghanistan                April  1961       -1.786
4       Afghanistan                  May  1961        1.351
...             ...                  ...   ...          ...
304159         OECD          DecJanFeb  2023        1.618
304160         OECD          MarAprMay  2023        1.210
304161         OECD          JunJulAug  2023        1.743
304162         OECD          SepOctNov  2023        2.254
304163         OECD  Meteorological year  2023        1.706

[304164 rows x 4 columns]


In [9]:
melted_df = melted_df.rename(columns={'Area': 'Country'})

# Reset index to sequence
melted_df.reset_index(drop=True, inplace=True)

# Display the DataFrame with modified column name and reset index
print(melted_df)

            Country               Months  Year  Temperature
0       Afghanistan              January  1961        0.745
1       Afghanistan             February  1961       -1.787
2       Afghanistan                March  1961        0.445
3       Afghanistan                April  1961       -1.786
4       Afghanistan                  May  1961        1.351
...             ...                  ...   ...          ...
304159         OECD          DecJanFeb  2023        1.618
304160         OECD          MarAprMay  2023        1.210
304161         OECD          JunJulAug  2023        1.743
304162         OECD          SepOctNov  2023        2.254
304163         OECD  Meteorological year  2023        1.706

[304164 rows x 4 columns]


In [10]:
valid_months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

# Filter out rows with months not in the list of valid months
filtered_df = melted_df[melted_df['Months'].isin(valid_months)]
filtered_df.reset_index(drop=True, inplace=True)
# Display the filtered DataFrame
print(filtered_df)

            Country     Months  Year  Temperature
0       Afghanistan    January  1961        0.745
1       Afghanistan   February  1961       -1.787
2       Afghanistan      March  1961        0.445
3       Afghanistan      April  1961       -1.786
4       Afghanistan        May  1961        1.351
...             ...        ...   ...          ...
214699         OECD     August  2023        1.906
214700         OECD  September  2023        2.589
214701         OECD    October  2023        1.957
214702         OECD   November  2023        2.216
214703         OECD   December  2023        4.275

[214704 rows x 4 columns]


In [11]:
filtered_df.to_csv('Temperature_change_Data_New.csv', index=False)

In [12]:
filtered_df1 = melted_df[melted_df['Months'].isin(['Meteorological year'])]
filtered_df1.reset_index(drop=True, inplace=True)
# Display the filtered DataFrame
print(filtered_df1)

                                       Country               Months  Year  \
0                                  Afghanistan  Meteorological year  1961   
1                                      Albania  Meteorological year  1961   
2                                      Algeria  Meteorological year  1961   
3                               American Samoa  Meteorological year  1961   
4                                      Andorra  Meteorological year  1961   
...                                        ...                  ...   ...   
17887        Low Income Food Deficit Countries  Meteorological year  2023   
17888  Net Food Importing Developing Countries  Meteorological year  2023   
17889                        Annex I countries  Meteorological year  2023   
17890                    Non-Annex I countries  Meteorological year  2023   
17891                                     OECD  Meteorological year  2023   

       Temperature  
0           -0.126  
1            0.635  
2           

In [13]:
filtered_df1.to_csv('Temperature_change_Data_year.csv', index=False)

In [14]:
filtered_df1['Year'] = pd.to_numeric(filtered_df1['Year'])
last_10_years = filtered_df1['Year'].max() - np.arange(10)
last_10_years_df = filtered_df1[filtered_df1['Year'].isin(last_10_years)]
sum_temperature_last_10_years = last_10_years_df.groupby('Country')['Temperature'].mean().reset_index()
print(sum_temperature_last_10_years)

            Country  Temperature
0       Afghanistan       1.3766
1            Africa       1.2327
2           Albania       1.6423
3           Algeria       1.6839
4    American Samoa       0.9816
..              ...          ...
279           World       1.4549
280           Yemen          NaN
281    Yugoslav SFR          NaN
282          Zambia       1.2548
283        Zimbabwe       0.5468

[284 rows x 2 columns]


In [15]:
sum_temperature_last_10_years.to_csv('Temperature_change_last_10_years.csv', index=False)